# COMP5318 - Machine Learning and Data Mining - S1 2018

## Assignment 1 - Due 07 May 2018, 5:00pm

Please refer to [Description File](https://drive.google.com/open?id=1fUFTqcjbSR75tiS48imwjlEVxZVS6U42YElwoVPexwY) for information about the assignment.



#### Group Number: 75
#### Group Members (name and student number):

*   Quang Trung Nguyen - 470518197
*   Mingxuan Li - 470325230
*   Yukui Chen - 470183984

#### Tutor Names: 
*   Kelvin & Anthony
*   Harrison Nguyen

#### Instructions to run the code:
*   Please click 'Run all' in the 'Runtime' if you want to run them all in sequence, or you can run each code cell in order. Please let us know if you have any questions about running the codes in the codelab.




---



## Authenticate and create PyDrive client.

You will be prompted with a link to click on, and give permission to Google Colab to access your Google Drive. If you don't want to give permission to your personal google drive, create a new gmail account, and complete this process using the new account.

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


## Import data
Locate the "Data" folder in your drive. Right click and click "share" to get the ID of the folder. Replace < Data folder id > with the id you got. (id should look like "1j8oG_vCmum965Ghg8LdbSkfj-lfi-AZ0" )

In [0]:
import os
import psutil
import numpy as np
import pandas as pd
import scipy.sparse as sparse


In [0]:
### Define Helper Function
### Reference: http://fa.bianp.net/blog/2013/different-ways-to-get-memory-consumption-or-lessons-learned-from-memory_profiler/
def memory_usage_psutil():
    # return the memory usage in MB
    process = psutil.Process(os.getpid())
    mem = process.memory_info().rss / float(2 ** 20)
    return mem

In [28]:
file_list = drive.ListFile({'q': "'1j8oG_vCmum9YuVWP8LdbSkfj-lfi-AZ0' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: training_desc.csv, id: 1kSoeoaK_TsTHnhSGPXOLNy43jd20G8bH
title: training_data.csv, id: 12-B9GpferXatOB4ACo7GanQ2iBt5O-66
title: test_data.csv, id: 1y_YySFyibYYwQtJr6v7Qf81FqULwK7Fk
title: training_labels.csv, id: 1cqErOBjBB91P_xrt4TKuZ91OfTLoUYmx


### Pulling data into Google Colab.

In [0]:
training_data_downloaded = drive.CreateFile({'id': '12-B9GpferXatOB4ACo7GanQ2iBt5O-66'})
training_data_downloaded.GetContentFile('training_data.csv')

training_desc_downloaded = drive.CreateFile({'id': '1kSoeoaK_TsTHnhSGPXOLNy43jd20G8bH'})
training_desc_downloaded.GetContentFile('training_desc.csv')

training_labels_downloaded = drive.CreateFile({'id': '1cqErOBjBB91P_xrt4TKuZ91OfTLoUYmx'})
training_labels_downloaded.GetContentFile('training_labels.csv')

test_data_downloaded = drive.CreateFile({'id': '1y_YySFyibYYwQtJr6v7Qf81FqULwK7Fk'})
test_data_downloaded.GetContentFile('test_data.csv') 

### Load data files
(example:)

In [0]:
### Define Data Dimensions
train_data_dimension = np.array([20104, 13627])
test_data_dimension = np.array([2233, 13627])
usecolumns = np.arange(1, train_data_dimension[1], 1)

In [0]:
### Load Files
### Reference - Read CSV using Pandas: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
### Reference - Load Big Data using Pandas: https://www.dataquest.io/blog/pandas-big-data/
df_train_names = pd.read_csv('training_data.csv', sep=',', engine='c', usecols=[0], header=None, names=['app_name'])
df_train_data = pd.read_csv('training_data.csv', sep=',', engine='c', dtype=np.float32, usecols=usecolumns, header=None)
df_test_names = pd.read_csv('test_data.csv', sep=',', engine='c', usecols=[0], header=None, names=['app_name'])
df_test_data = pd.read_csv('test_data.csv', sep=',', engine='c', dtype=np.float32, usecols=usecolumns, header=None)
df_train_labels = pd.read_csv('training_labels.csv', sep=',', engine='c', header=None, names=['app_name', 'app_label'])
# df_desc = pd.read_csv('training_desc.csv', sep=',', engine='c', header=None)



---


## 1. Introduction (max 500 words)


### 1.1 What is the aim of the study?



We have three aims for this study. The first aim is to find an effective classification method that can be applied to the problem of app classification and to build an appropriate classifier to tag apps with good accuracy and minimal running time into the correct label based on the app’s description. The dataset introduces TF-IDF values to represent the importance of a word in an App's description. As there are 13,626 unique words which correspond to their frequency in the description, we treat them as 13,626 features. Therefore, the classification method we will choose should be suitable for dealing with high-dimensioned data so that it can run within feasible time.

</br>

The second aim of this study is to learn about the working principle of the Naive Bayesian classifier which is the approach we choose to solve the problem of app classification. The Naive Bayesian approach is good at text classification. It is a probabilistic classifier with the assumption of independency among features, and this assumption is helpful when the dimensionality of the input data is high (Bishop, 2006). As the number of feature dimensions increases, the total number of possible feature vectors increases exponentially. This severe difficulty that may arise from moving to spaces of higher dimension was termed the curse of dimensionality by Richard Bellman (1961). Text classification is such a domain with a large number of features (McCallum and Nigam 1998).  In this case, we will consider Naive Bayes classifier which assumes that all attributes of the data are independent of each other given the context of the class.

</br>

The third aim of this study is to judge and criticize the classifier we build in terms of accuracy, running time, memory usage and ultimately evaluate the applicability of Naïve Bays approach in this problem. As limitation are a part of the scientific study, we provide an explanation for each limitation, showing why the classification results are important.




###1.2 Why is this study important?

There are some reasons that drive our team to undertake this classification task. 
Firstly, in this particular case of App classification, more classification tasks in App market can be efficiently processed by a machine learning based classifier which simply learns from training data of the App's TD-IDF value. As the widespread and increasing availability of mobile Apps in the Apps Markets presents significant challenges to sort the Apps into an appropriate category, in order to make it easier for Apps users to search and download the apps, developing a better classifier powered by machine learning algorithm helps the market reduce cost and save more time for the app development community and ultimately enhances user experience.

</br>
Moreover, in general, an effective classification method allows efficient study in many fields, for both commercial and academic purposes. For example, in biological study, if the organisms can be categorized based on their characteristics, ancestry and so on, the researchers will have the ability to conduct deeper learning on them and even possibly extract some meaningful insights and patterns which almost impossible to recognize without an effective classification.




## 2. Methods 

### 2.1 Pre-processing


---



The pre-processing before we build the classifier includes three parts which are (1) Load data efficiently and using less memory, (2) Construct training data and testing data into a sparse matrix and (3) Join the training data and training labels in the same orders by the app_name. Firstly, we use pandas to read the csv file into dataframes types, we use C engine to achieve faster data loading, we also load the training data into ***df_train_names*** and ***df_train_data*** separately, so ***df_train_names*** are strings and will be used in joining with training labels in same order in step 3 as mentioned above. The actual training data and testing data are loaded in *np.float32* type to save 50% more memory while loading as by default *np.float64*.

</br>

After loading the data of csv file into dataframes, we use *np.nonzero* function to find all the non-zero indices in the training and testing matrix, then we use spicy sparse matrix library and those non-zero indices to construct two sparse matrices (***sp_matrix_train*** and ***sp_matrix_test***), then we delete those dataframes from step 1 to release those memories for garbage collection (GC) to collect at later stage. Once the training and testing sparse matrices is constructed, we join the labels into the same order as the app_names (first column) in the training data set so we can map the labels into the training data properly for later classifiers and predictions steps.


</br>


The pre-processing steps help us save much more memories. In the local environment we use around 3GB without pre-processing, while using pre-processing we only use around 500MB. Similarly, in google colab platform, we actually spend *4649.234375MB* in total by using preprocessing steps as described above without affecting the overall runtime and accessibilities of the sparse matrices.


In [32]:
### Reshape the data
### Convert DataFrame to Matrix
train_data = df_train_data.astype(np.float32).as_matrix()
test_data = df_test_data.astype(np.float32).as_matrix()

### Compress Sparse Matrix
sp_train_indices = np.nonzero(train_data)
sp_train_data = train_data[sp_train_indices].astype(np.float32)
sp_test_indices = np.nonzero(test_data)
sp_test_data = test_data[sp_test_indices].astype(np.float32)

sp_matrix_train = sparse.csc_matrix((sp_train_data, (sp_train_indices[0], sp_train_indices[1])), shape=df_train_data.shape)
sp_matrix_test = sparse.csc_matrix((sp_test_data, (sp_test_indices[0], sp_test_indices[1])), shape=df_test_data.shape)
print(sp_matrix_train.shape)
print(sp_matrix_test.shape)

### Delete those dataframes to give to gc to recycle memory as we dont need them for classifier and data analysis below
del df_train_data
del df_test_data
del train_data
del test_data

df_train_mapped_names_labels = df_train_names.join(df_train_labels.set_index('app_name'), on='app_name')
df_train_unique_labels = df_train_mapped_names_labels.app_label.unique()
df_train_unique_labels_size = df_train_unique_labels.size

(20104, 13626)
(2233, 13626)


### 2.2 Classifier


---



aAccording to McCallum and Nigam (1998), text classification is such a domain with a large number of features. In this project, word frequency in app’s description can be treated as the word counts in text and the fractional counts TF-IDF value can be seen as integer feature counts. By consider Naive Bayes classifier, we assume that all attributes of the data are independent of each other given the context of the class. Besides a large number of features, we need to consider other factors like easy and fast implementation. The training time will be a big concern because our training data consists 20,104 apps with 13,626 features and the task are to classify them into 30 unique labels. This suggests Naive Bayes classifier normally to be a good technique used as a baseline in text classification. Naive Bayes classifier requires less model training time due to its simplicity especially for the classification tasks of text data with high dimensions. According to Statista (2017), the current growth rate of mobile apps in the market is more than 1,300 apps per day, which means the input data in Apps Markets are dynamic, Naive Bayes classifier can adapt quickly to the changing data due to its simple and fast algorithm. 

</br>
 
Under the Bayesian probabilistic framework, Naive Bayes approach uses the training data which is assumed to be generated by a parametric model to compute Bayes-optimal estimates of the model parameters (McCallum and Nigam 1998). With the classifier that is produced by the training data, we can use it to classify new test data via Bayes’ rule to turn the generative model around and compute the posterior probability that a class will generate on the test examples. Then, any new app will be tagged into the most probable label at a certain accuracy that our classifier suggests.

</br>

Based on Bayes' theorem, to predict class C, specifically, we need to find the value of C that maximizes  $P(C | A1, A2,..., An )$. As we want to calculate the posterior probability $P(C | A1, A2, ..., An)$ for all values of C using the Bayes' theorem, which is equivalent to select value of C that maximises $P(A1, A2, ..., An | C) P(C) $. We will use Naive Bayes classifier to estimate $P(A1, A2, ..., An | C )$. 

</br>

$$ P(C \mid A) = \frac{P(A \mid C) \, P(C)}{P(A)} $$

</br>

In Naive Bayes approach, we assume the feature probabilities $P(A_i|C_j)$ are independent given the class C.

</br>

$$ P({A_1,A_2...A_n} \mid C) = P(A_1 \mid C) \times P( A_2 \mid C)\times...\times P( A_n \mid C) $$

</br>

The Naïve Bayes classifier (NBC) is given by,
\begin{equation}
    C_{NB} = \text{argmax}_{k \in {1,2,...K}} \Big( p(C_k) \prod_{i=1}^D p(x_i|C_k) \Big) \in \{C_k\}_{k=1}^K
\end{equation}

where $C_k$ is the $30^{th}$ class when there are $30$ classes. 

</br>

The general Naive Bayes approach refers strong independence assumptions in the model, instead of the particular distribution of each feature. By considering the fact that the TF-IDF value, which is used to weigh each word in the app's description according to its uniqueness, is essentially a calculation of a discrete feature to present the word counts, we will implement the Multinomial Naive Bayes model for classification. Russel and Norvig (2003) also supported that multinomial Naive Bayes works well for data which can easily be turned into counts, such as word frequency in text. To be more specifically, Multinomial Naïve Bayes classifier is a special form of a Naïve Bayes classifier that assumes the multinomial distribution for each feature in the model. 


\begin{align}
\log p(C_k \mid \mathbf{x}) & \varpropto \log \left( p(C_k) \prod_{i=1}^n {p_{ki}}^{x_i} \right) = \log p(C_k) + \sum_{i=1}^n x_i \cdot \log p_{ki}
\end{align}


</br>


The multinomial naive Bayes classifier is basically a linear classifier when represented in log-space. Therefore, we take the log of each probability of each class then sum them up to get the log form of posterior probability of the class.
</br></br>

To implement the model, we defined two functions, one for calculating the feature probabilities, and the other for predicting the class probabilities given the features. The first function ***multinomial_Bayes_classifier*** takes in three parameters: train data, train labels and feature count. For each class label *k*, we first retrieve all rows in the train data belonging to class *k*, then calculate the log-probability $Pki$ of each feature *i* given that class *k*, store the values in a matrix and return it to the caller. The next function ***predict_labels*** takes in four parameters: test data, class label count, feature probabilities, and class probabilities. This function calculates the log-probability $log p(C_k \mid \mathbf{x})$ for each class *k* given the set of features in the test data using  $\log p(C_k) + \sum_{i=1}^n x_i \cdot \log p_{ki}$ where $x_i$ is the TF-IDF values in the test data, then select the class with the highest log-probability to be the predicted label.

In [0]:
### MULTINOMAL NAIVE BAYES MODEL 

def multinomial_Bayes_classifier(train_data, x_train_labels, feature_count):
    x_train_labels_counts = x_train_labels.value_counts()
    feature_probabilities = np.empty((0, feature_count), dtype=np.float32)
    # For each class label, calculate each feature probability given that class
    for labelname, count in x_train_labels_counts.items():
        # Get the data of each class
        class_rows_indices = x_train_labels[x_train_labels == labelname].index
        class_data = train_data[class_rows_indices, :]
        # Calculate feature probabilities 
        feature_prob = np.log(class_data.sum(axis=0) + 1)
        feature_probabilities = np.append(feature_probabilities, feature_prob[0], axis=0)
    return feature_probabilities


In [0]:
### Function to Predict Labels

def predict_labels(test_data, feature_probabilities, x_train_labels_counts, x_train_class_probabilities):
    fold_size = test_data.shape[0]
    # Reshape class probablities into a matrix
    x_train_class_probabilities_matrix = x_train_class_probabilities.repeat(fold_size, axis=0).reshape(fold_size, x_train_labels_counts.size)
    # Calculate class probabilities given feature set and select the class with the highest probability
    predicted_label_indices = np.argmax(test_data * feature_probabilities.T + x_train_class_probabilities_matrix, axis=1)
    predicted_results = []
    # Convert numerical class label into respective text label
    for labelindex in np.squeeze(np.asarray(predicted_label_indices)):
        predicted_results.append(x_train_labels_counts.keys().values[labelindex])
    return np.asarray(predicted_results)

## 3. Experiments and Results 

We defined two functions, ***calculate_performance_metrics*** for calculating the performance measures on the result of our classifier, and ***k_fold_cross_validation*** for dividing the dataset into k folds for cross validation. The ratio used is 9:1 for train and test sets in each iteration.
</br> </br>
We measured the performance of our classifier not only by accuracy but also using precision, recall, f-score. We calculated these three performance metrics for each class as we do with binary classification, then we averaged them for the entire dataset.

In [0]:
### Function to calculate performnance metrics

def calculate_performance_metrics(y_true, y_pred):
  TP = [] # True Positive
  TN = [] # True Negative
  FP = [] # False Positive
  FN = [] # True Negative
  
  # Calculate the precision, recall, f-measure for each class label
  for label in df_train_unique_labels:
    true = np.copy(y_true)
    pred = np.copy(y_pred)
    # Relabel the data as binary classification 
    true[true != label] = 'none'
    pred[pred != label] = 'none'
    TN.append(len(np.where((pred != label) & (true == pred))[0]))
    TP.append(len(np.where((pred == label) & (true == pred))[0]))
    FP.append(len(np.where((pred == label) & (true != pred))[0]))
    FN.append(len(np.where((pred != label) & (true != pred))[0]))
    
  # Precision = TP/(TP+FP)
  precision = np.asarray(TP) / ( np.asarray(TP) + np.asarray(FP)) 
  # Recall = TP/(TP+FN)
  recall = np.asarray(TP) / ( np.asarray(TP) + np.asarray(FN) ) 
  # F-measure = 2TP/(2TP+FN+FP)
  f_measure = 2*np.asarray(TP) / ( 2*np.asarray(TP) + np.asarray(FN) + np.asarray(FP) )
  
  # Return the mean scores across all classes
  return np.mean(precision),np.mean(recall),np.mean(f_measure)


In [0]:
### Functions for k-fold cross validation

def k_fold_cross_validation(train_data, train_mapped_names_labels, k):
    sample_size = train_data.shape[0]
    feature_count = train_data.shape[1]
    fold_size = int(sample_size / k)
    # Performance metrics
    accuracies = []
    precisions = []
    recalls = []
    f_measures = []
    
    ### Divide the dataset into k folds
    for start in np.arange(0, sample_size, fold_size):
        stop = start + fold_size
        if stop > sample_size:
            break

        ### For each fold we find the corresponding training data set and testing data set with ratio (9:1)
        x_test_indices = np.arange(start, stop, 1)
        x_test_data = train_data[x_test_indices]
        x_test_labels = train_mapped_names_labels['app_label'][x_test_indices]
        x_train_indices = np.append(np.arange(0, start, 1), np.arange(stop, sample_size, 1))
        x_train_data = train_data[x_train_indices]
        x_train_labels = train_mapped_names_labels['app_label'][x_train_indices]
        x_train_labels_counts = x_train_labels.value_counts()
        
        # Calculate class probabilities (priors) from the train data
        x_train_class_probabilities = np.log(x_train_labels_counts / x_train_labels.size).values
        # Fit the train data to calculate feature probabilities
        feature_probs = multinomial_Bayes_classifier(train_data, x_train_labels, feature_count)    
        # Predict labels for test data 
        predicted_result = predict_labels(x_test_data, feature_probs, x_train_labels_counts, x_train_class_probabilities) 
        
        # Calculate accuracy, performance metrics
        accuracies.append(sum(np.asarray(x_test_labels) == predicted_result) / fold_size)
        precision,recall,f_measure = calculate_performance_metrics(np.asarray(x_test_labels),predicted_result)
        precisions.append(precision)
        recalls.append(recall)
        f_measures.append(f_measure)
        
    # Calculate and print the mean, standard deviation of each metric
    print('Mean Accuracy: {:0.2f}%'.format(np.mean(np.asarray(accuracies))*100))
    print('Accuracy Std Dev: {:0.2f}%'.format(np.std(np.asarray(accuracies))*100))
    print(accuracies)
    print('Mean Precision: {:0.3f}'.format(np.mean(np.asarray(precisions))))
    print('Precision Std Dev: {:0.3f}'.format(np.std(np.asarray(precisions))))
    print(precisions)
    print('Mean Recall: {:0.3f}'.format(np.mean(np.asarray(recalls))))
    print('Recall Std Dev: {:0.3f}'.format(np.std(np.asarray(recalls))))
    print(recalls)
    print('Mean F-measure: {:0.3f}'.format(np.mean(np.asarray(f_measures))))
    print('F-measure Std Dev: {:0.3f}'.format(np.std(np.asarray(f_measures))))
    print(f_measures)

### 3.1 Accuracy

We performed 10-fold cross validation and take average of the accuracy across 10 iterations and finally get a mean accuracy of 63.60%. The standard deviation of the accuracies is very low - 0.67%. This indicates there are only small differences in the accuracies over 10 iterations, and the mean average is a good representation of the accuracy over the entire dataset. 

In [37]:
# Timing
import timeit
start = timeit.default_timer()

# Perform 10-fold cross validation
# This should take roughly 1.5 minutes
k = 10
k_fold_cross_validation(sp_matrix_train, df_train_mapped_names_labels, k)

stop = timeit.default_timer()
print ('Runtime: {:0.2f} seconds'.format(stop - start ))

Mean Accuracy: 63.60%
Accuracy Std Dev: 0.67%
[0.6437810945273632, 0.6308457711442786, 0.6358208955223881, 0.627363184079602, 0.6318407960199005, 0.6388059701492538, 0.6373134328358209, 0.6497512437810945, 0.6278606965174129, 0.6368159203980099]
Mean Precision: 0.660
Precision Std Dev: 0.011
[0.6765124115507664, 0.6493054171874849, 0.6629803664475552, 0.650369218983579, 0.6544886671532452, 0.6615587074556031, 0.6628238056706273, 0.6802343660864627, 0.6418735824328974, 0.6553218255196313]
Mean Recall: 0.625
Recall Std Dev: 0.005
[0.6315976337658676, 0.6189976877408248, 0.624350648170524, 0.6214385055806506, 0.6202311268591459, 0.6243804255720945, 0.6244866700573405, 0.6313506045939948, 0.6201518215703296, 0.6332611325052198]
Mean F-measure: 0.618
F-measure Std Dev: 0.006
[0.6255821136543153, 0.6138708372930722, 0.6172074561103998, 0.6128873423635759, 0.612648718083106, 0.6184662872032415, 0.618521349418539, 0.6306453301073985, 0.6080642560802045, 0.6224000691023821]
Runtime: 83.74 secon

We then compared the result of our classifier with scikit-learn's Multinomial Naive Bayes classifier and its k-fold function. The accuracy from their classifier using 10-fold cross validation on our train data is very similar to the accuracy of our own classifier (63.23% and 63.60% respectively).

In [38]:
# Test our data with scikit-learn MultinomialNB using 10-fold cross validation

from sklearn.naive_bayes import MultinomialNB 
from sklearn.model_selection import KFold
k_folds = KFold(n_splits=10)
accuracy = []
for train, test in k_folds.split(sp_matrix_train):
  train_data = sp_matrix_train[train]
  test_data = sp_matrix_train[test]
  train_label = df_train_mapped_names_labels['app_label'][train]
  test_label = df_train_mapped_names_labels['app_label'][test]
  NB = MultinomialNB()
  NB.fit(train_data, train_label)
  y_pred = NB.predict(test_data)
  accuracy.append(sum(np.asarray(test_label) == y_pred) / test_data.shape[0])

print('Scikit-learns MultinomialNB 10-fold cross validation accruacy: {:0.2f}%'.format(np.mean(np.asarray(accuracy))*100))


Scikit-learns MultinomialNB 10-fold cross validation accruacy: 63.23%


### 3.2 Extensive Analysis



The accuracy is a great measure but only when we have symmetric datasets where values of false positive and false negatives are almost same. We computed the performance metrics and got the average precision, recall and F-measure across all class labels as 65.96%, 62.50% and 61.80%. respectively. The values are very similar to each other, and also to our mean accuracy of 63.60%, which shows that the number of false positive and false negative is quite balanced in our dataset. The metrics are suggest that our mean accuracy is a non-biased measure of our classifier. 

</br>

\begin{equation}
    Accuracy = \frac{TP+TN}{TP+TN+FP+FN}  
\end{equation}

</br>

\begin{equation}
    Precision (p) = \frac{TP}{TP+FP}, 
    Recall (r) = \frac{TP}{TP+FN},
    F\_measure (F) = \frac{2rp}{r+p} = \frac{2TP}{2TP+FN+FP}
\end{equation}

</br>


We suspect there might be imbalance in the number of samples among class labels so instead of calculating these metrics for each class then macro-averaged them for the entire dataset, we considered micro-averaging the performance metrics by taking the mean of the TP,TN,FP,FN values of each class label, then using these mean values to calculate the overall performance metrics. The results for precision, recall and f-measure are identical, however they are similar to the results we got above when we macro-averaged them. This confirms that number of samples is quite balanced for all class labels.



</br>
\begin{equation}
Precision_{micro} = \frac{TP_1 + ... + TP_k  }{TP_1 + ... + TP_k +FP_1 + ... + FP_k }, 
\end{equation}
</br>

</br>
\begin{equation}
Precision_{macro} = \frac{Precision_1 + ... + Precision_k  }{k}
\end{equation}

</br>



In [39]:
### Obtain training accuracy
feature_count = sp_matrix_train.shape[1]
sample_size = sp_matrix_train.shape[0]
x_train_labels_counts = df_train_mapped_names_labels['app_label'].value_counts()
x_train_labels_probabilities = np.log(x_train_labels_counts / df_train_mapped_names_labels['app_label'].size).values
classess_feature_probs = multinomial_Bayes_classifier(sp_matrix_train, df_train_mapped_names_labels['app_label'], feature_count)  
predicted_result = predict_labels(sp_matrix_train, classess_feature_probs, x_train_labels_counts, x_train_labels_probabilities) 
print('Training accuracy: {:0.2f}%'.format(sum(np.asarray(df_train_mapped_names_labels['app_label']) == predicted_result) / sample_size * 100))


Training accuracy: 73.73%



In order to see if our classifier overfits with the training data, we trained and tested our classifier with the same dataset. The training accuracy we obtained is 73.73%, which is slightly higher than our average test accuracy of 63.60% using 10-fold cross validation. There is a no strong sign to show that our classifier suffers from overfitting.



## 4. Export Results 

#### The 'predicted_labels.csv' file will be exported to the same folder under 'Data': https://drive.google.com/open?id=1OiK2KB7aonQVzKE6dDA0Vnyk3Y190jpi

In [40]:
# Timing
start = timeit.default_timer()

### Predict Labels and Save to PyDrive
#### Code to predict labels
train_labels = df_train_mapped_names_labels['app_label']
train_labels_counts = train_labels.value_counts()
train_labels_probabilities = np.log(train_labels_counts / train_labels.size).values
classess_feature_probs = multinomial_Bayes_classifier(sp_matrix_train, train_labels, feature_count)
predicted_labels = predict_labels(sp_matrix_test, classess_feature_probs, train_labels_counts, train_labels_probabilities) 
predicted_result = pd.concat([df_test_names, pd.DataFrame(predicted_labels, columns=['predicted_labels'])], axis=1)

### Generate the predicted_labels.csv to the same Data folder: https://drive.google.com/open?id=1OiK2KB7aonQVzKE6dDA0Vnyk3Y190jpi
tgt_folder_id = '1OiK2KB7aonQVzKE6dDA0Vnyk3Y190jpi'
parent_folder = {"kind": "drive#fileLink","id": tgt_folder_id}
predicted_labels = drive.CreateFile({'title': 'predicted_labels.csv', 'mimeType':'text/csv', 'parents': [parent_folder]})
predicted_labels.SetContentString(predicted_result.to_csv(index=False, header=False))
predicted_labels.Upload()
print('Created file %s with mimeType %s' % (predicted_labels['title'], predicted_labels['mimeType']))  
print(predicted_result)

stop = timeit.default_timer()
print ('Runtime: {:0.2f} seconds'.format(stop - start ))

Created file predicted_labels.csv with mimeType text/csv
                                            app_name     predicted_labels
0               dalmax.games.turnBasedGames.connect4     Brain and Puzzle
1                       com.holfeld.japaneseplusfree     Travel and Local
2                           com.mobileApp.controller      Personalization
3                   com.aarontennyson.calorietracker   Health and Fitness
4               com.totaldevel.android.todocitas.ads  Books and Reference
5                     com.google.android.voicesearch        Communication
6                                 fieldbird.yourself            Lifestyle
7                          com.dmacattack.mpgConvert       Transportation
8                               com.ds.tonsillectomy              Medical
9                         metrofax.android.MobileFax         Productivity
10                                 sk.halmi.itimerad   Health and Fitness
11               com.dreamstep.wPureRomancebyAleacia   

In [41]:
print('Memory usage for this process so far is: ' + str(memory_usage_psutil()) + ' MB')

Memory usage for this process so far is: 5705.1171875 MB


**Insert the url address of your predicted_labels.csv file:   **



https://drive.google.com/open?id=1-GqSDX8aA0vUXMtl6En5lH0W4w4sU_5h

## 5. Discussion 


---




### 5.1	Runtime and Memory Usage

Our classifier takes approximately 85 seconds to run the 10-fold cross validation on our training dataset, and 11 seconds to predict the labels for our test dataset. After we run all the codes, the total memory usage is roughly 4658 MB. Hardware and software specifications of our local computer: 
MacOS High Sierra, Processor: 2.7 GHz Intel Core i7, Memory: 16 GB, Graphics: Radeon Pro 455 2048 MB, Intel HD Graphics 530 1536 MB. IDE: Pycharm.



### 5.2	Problem with Naïve Bayes classifier

The major systemic problem with Naive Bayes approach is that the independent assumption of all the feature is drastic and rarely true. As each app has 13,626 features, there is a high probability that some of the features has covariate effect with each other and the assumption of independent features may fail to hold. Furthermore, there is no feature name in the dataset, the uncertainty about the feature exacerbated the problem of independent assumption. Another problem with Naive Bayes Classifier is that the requirement of balanced training samples.  The app's data that corresponding to these 30 classes are relatively balanced, but not perfectly balance. In this case, the unbalanced effect under different labels may warn us that if the class has more training examples than another, Naive Bayes classifier may select poor weights for the decision boundary. This is due to an understudied bias effect that shrinks weights for classes with few training examples (Rennie et.al., 2016). However, by using both micro-average and macro-average to compute and compare the performance metrics (recall, precision, f-measure) in our 10-fold cross validation, we are positive that the number of samples among 30 class labels in each iteration is quite balanced.



### 5.3 The drawback of the generative approach 

In general, the generative model is not good at optimizing classification accuracy. As a generative model, Naive Bayes approach aims for a complete probabilistic description of the data by constructing the joint probability distribution. However, our main focus is to optimize classification accuracy for Apps instead of making a solid claim about how model parameters interact. If we apply a discriminative model like SVM and logistic regression, the accuracy may be higher. 



## 6. Conclusion and future work 







---



By considering there will be more apps coming into the App market every day, we believe that Naïve Bayes classifier is stable and able to adapt quickly to the changes of dynamic data. Even though the “naïve” assumption of independence among features are hardly true, but in practice, Naive Bayes models have performed surprisingly well, even on complex tasks where the strong independence assumptions are false. Basically, some of the reasons the Naïve Bayes classifier is so common to use is that it is fast, easy to implement and relatively effective to undertake the classification tasks. However, by considering the size of feature and classes, it may not always work well. For performance evaluation, it is important to consider the class distribution and the costs of each misclassification in each class. The future work of this problem could be discussed by introducing incremental learning algorithm. Also, to improve the classification accuracy, we might consider non-linear classifiers and other discriminative models such SVM and logistic regression.

## 7. References


---




1.   Pandas.read_csv. (n.d.). Retrieved 25 April 2018 from https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
2.   DataQuest. (2017). Using pandas with large data. Retrieved 25 April 2018 from https://www.dataquest.io/blog/pandas-big-data/
3.   Stackoverflow. (2014). Retrieved 25 April 2018 from https://stackoverflow.com/questions/22934525/pydrive-cannot-write-file-to-specific-gdrive-folder
4.   Stuart, J., Russell, P and Norvig, P. (2003). *Artificial Intelligence: A Modern Approach (2 ed.). Pearson Education.* See p. 499 for reference to "idiot Bayes" as well as the general definition of the Naive Bayes model and its independence assumptions
5. Domingos, P., & Pazzani, M. (1996). Beyond independence: conditions for the optimality of the simple Bayesian classifier. Proceedings of ICML ’96. 

6. McCallum, A., & Nigam, K. (1998). *A comparison of event models for naive Bayes text classification. Proceedings of AAAI* . 
7. Rennie, J., Shih, L. & Teevan, J. (2003). *Tackling the Poor Assumptions of Naive Bayes Text Classifiers*.Artificial Intelligence Laboratory, Massachusetts Institute of Technology







# Sumbitting your assignment

You must share only 1 folder with your tutor.
The folder should have your group's unikeys in its name ( example: Assignment1_abxy6273_edfr3373_yhfr4534 ).

The folder should contain a Colab notebook and the output file.

Suggested structure of the folder:

* Data
 * training_data.csv
 * training_desc.csv
 * training_labels.csv
 * test_data.csv
* Assignment1.ipynb
* predicted_labels.csv

Ask your tutor for his email address and share the properly named folder with him before 5:00pm on May 7th 2018. 
Every late day will cost you 20 marks.

